In [35]:
!pip install optuna

In [36]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [37]:
df.shape

(768, 9)

In [38]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column

imputer = simple_imputer = SimpleImputer(strategy='mean')
df[cols_with_missing_vals] = imputer.fit_transform(df[cols_with_missing_vals])

# imputer = IterativeImputer(random_state=42)
# df[cols_with_missing_vals] = imputer.fit_transform(df[cols_with_missing_vals])

# df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [39]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.00000,155.548223,33.6,0.627,50,1
1,1,85.0,66.0,29.00000,155.548223,26.6,0.351,31,0
2,8,183.0,64.0,29.15342,155.548223,23.3,0.672,32,1
3,1,89.0,66.0,23.00000,94.000000,28.1,0.167,21,0
4,0,137.0,40.0,35.00000,168.000000,43.1,2.288,33,1


In [40]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [42]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-02-20 17:44:04,130] A new study created in memory with name: no-name-856f0d71-1939-47d8-94fb-b254bd4a192e
[I 2025-02-20 17:44:06,983] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 170, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-02-20 17:44:07,809] Trial 1 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 97, 'max_depth': 4}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-02-20 17:44:08,727] Trial 2 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 124, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-02-20 17:44:09,533] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 91, 'max_depth': 12}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-02-20 17:44:10,801] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 121, 'max_depth': 14}. Best is trial 0 with value: 0.76908752

In [44]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 115, 'max_depth': 15}


In [21]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


## Samplers in Optuna

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [23]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-02-20 17:41:31,937] A new study created in memory with name: no-name-b5ac60ba-5add-4264-9163-a6eb11755d3d
[I 2025-02-20 17:41:32,235] Trial 0 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 69, 'max_depth': 4}. Best is trial 0 with value: 0.7635009310986964.
[I 2025-02-20 17:41:32,506] Trial 1 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 53, 'max_depth': 18}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-02-20 17:41:33,222] Trial 2 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 170, 'max_depth': 4}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-02-20 17:41:34,331] Trial 3 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 194, 'max_depth': 6}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-02-20 17:41:36,112] Trial 4 finished with value: 0.7765363128491621 and parameters: {'n_estimators': 199, 'max_depth': 19}. Best is trial 4 with value: 0.776536312

In [24]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7783985102420857
Best hyperparameters: {'n_estimators': 71, 'max_depth': 20}


In [25]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


In [26]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [27]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-02-20 17:42:13,106] A new study created in memory with name: no-name-d2412a33-6812-48bc-9ae8-bd5d9a2313ff
[I 2025-02-20 17:42:13,538] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-02-20 17:42:14,227] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-02-20 17:42:14,461] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-02-20 17:42:14,933] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-02-20 17:42:15,407] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [28]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [29]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [30]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [45]:
# 1. Optimization History
plot_optimization_history(study).show()

In [46]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [47]:
# 3. Slice Plot
plot_slice(study).show()

In [48]:
# 4. Contour Plot
plot_contour(study).show()

In [49]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [50]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [51]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [52]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-02-20 17:46:45,449] A new study created in memory with name: no-name-d0fe0424-19ff-42ed-98b1-579951efe9f7
[I 2025-02-20 17:46:45,495] Trial 0 finished with value: 0.7858472998137801 and parameters: {'classifier': 'SVM', 'C': 6.030113532157219, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 0 with value: 0.7858472998137801.
[I 2025-02-20 17:46:45,925] Trial 1 finished with value: 0.7672253258845437 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 52, 'learning_rate': 0.0390332899063806, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.7858472998137801.
[I 2025-02-20 17:46:46,308] Trial 2 finished with value: 0.7597765363128491 and parameters: {'classifier': 'RandomForest', 'n_estimators': 90, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 2, 'bootstrap': False}. Best is trial 0 with value: 0.7858472998137801.
[I 2025-02-20 17:46:50,112] Trial 3 finished with value: 0.7374301675977654 and paramete

In [53]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'RandomForest', 'n_estimators': 105, 'max_depth': 15, 'min_samples_split': 9, 'min_samples_leaf': 9, 'bootstrap': False}
Best trial accuracy: 0.7858472998137803


In [54]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.785847,2025-02-20 17:46:45.450999,2025-02-20 17:46:45.494865,0 days 00:00:00.043866,6.030114,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.767225,2025-02-20 17:46:45.495752,2025-02-20 17:46:45.925217,0 days 00:00:00.429465,NaN,NaN,GradientBoosting,NaN,NaN,0.039033,20.0,10.0,6.0,52.0,COMPLETE
2,2,0.759777,2025-02-20 17:46:45.926162,2025-02-20 17:46:46.307902,0 days 00:00:00.381740,NaN,False,RandomForest,NaN,NaN,NaN,20.0,2.0,6.0,90.0,COMPLETE
3,3,0.737430,2025-02-20 17:46:46.308881,2025-02-20 17:46:50.112295,0 days 00:00:03.803414,NaN,NaN,GradientBoosting,NaN,NaN,0.249570,17.0,7.0,8.0,269.0,COMPLETE
4,4,0.756052,2025-02-20 17:46:50.118919,2025-02-20 17:46:50.942786,0 days 00:00:00.823867,NaN,True,RandomForest,NaN,NaN,NaN,20.0,4.0,8.0,76.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.735568,2025-02-20 17:47:27.337506,2025-02-20 17:47:29.978543,0 days 00:00:02.641037,NaN,NaN,GradientBoosting,NaN,NaN,0.144687,15.0,8.0,6.0,264.0,COMPLETE
96,96,0.785847,2025-02-20 17:47:29.979528,2025-02-20 17:47:30.029296,0 days 00:00:00.049768,7.855719,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.785847,2025-02-20 17:47:30.030064,2025-02-20 17:47:30.146832,0 days 00:00:00.116768,30.419581,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.785847,2025-02-20 17:47:30.147566,2025-02-20 17:47:30.206225,0 days 00:00:00.058659,9.864169,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [55]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,70
RandomForest,19
GradientBoosting,11


In [56]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.754190
RandomForest,0.768303
SVM,0.770764


In [57]:
# 1. Optimization History
plot_optimization_history(study).show()

In [58]:
# 3. Slice Plot
plot_slice(study).show()

In [59]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [61]:
! pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 4.1 MB/s eta 0:00:00


In [62]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-02-20 17:49:35,096] A new study created in memory with name: no-name-eac7a8ca-f11b-4e5b-b1c3-86741cf7d420


[0]	train-mlogloss:1.08690	eval-mlogloss:1.08796
[1]	train-mlogloss:1.07528	eval-mlogloss:1.07534
[2]	train-mlogloss:1.06262	eval-mlogloss:1.06234
[3]	train-mlogloss:1.05051	eval-mlogloss:1.05067
[4]	train-mlogloss:1.03667	eval-mlogloss:1.03681
[5]	train-mlogloss:1.02333	eval-mlogloss:1.02354
[6]	train-mlogloss:1.00851	eval-mlogloss:1.00841
[7]	train-mlogloss:0.99663	eval-mlogloss:0.99623
[8]	train-mlogloss:0.98484	eval-mlogloss:0.98447
[9]	train-mlogloss:0.97171	eval-mlogloss:0.97011
[10]	train-mlogloss:0.96167	eval-mlogloss:0.96005
[11]	train-mlogloss:0.94819	eval-mlogloss:0.94570
[12]	train-mlogloss:0.93465	eval-mlogloss:0.93274
[13]	train-mlogloss:0.92465	eval-mlogloss:0.92263
[14]	train-mlogloss:0.91607	eval-mlogloss:0.91254
[15]	train-mlogloss:0.90301	eval-mlogloss:0.89903
[16]	train-mlogloss:0.89262	eval-mlogloss:0.88840
[17]	train-mlogloss:0.88360	eval-mlogloss:0.87925
[18]	train-mlogloss:0.87189	eval-mlogloss:0.86692
[19]	train-mlogloss:0.86073	eval-mlogloss:0.85525
[20]	train

[I 2025-02-20 17:49:37,401] Trial 0 finished with value: 0.9666666666666667 and parameters: {'lambda': 4.7421501627390993e-08, 'alpha': 4.807165433712406e-06, 'eta': 0.019323900184738667, 'gamma': 1.5377219588699263e-05, 'max_depth': 8, 'min_child_weight': 10, 'subsample': 0.45156198719091817, 'colsample_bytree': 0.9621388929485496}. Best is trial 0 with value: 0.9666666666666667.


[0]	train-mlogloss:0.88638	eval-mlogloss:0.87997
[1]	train-mlogloss:0.65490	eval-mlogloss:0.62910
[2]	train-mlogloss:0.50042	eval-mlogloss:0.46977
[3]	train-mlogloss:0.39013	eval-mlogloss:0.35540
[4]	train-mlogloss:0.32184	eval-mlogloss:0.27628
[5]	train-mlogloss:0.27191	eval-mlogloss:0.21937
[6]	train-mlogloss:0.24009	eval-mlogloss:0.18511
[7]	train-mlogloss:0.23032	eval-mlogloss:0.17425
[8]	train-mlogloss:0.22101	eval-mlogloss:0.16227
[9]	train-mlogloss:0.21426	eval-mlogloss:0.15097
[10]	train-mlogloss:0.21413	eval-mlogloss:0.15183
[11]	train-mlogloss:0.20915	eval-mlogloss:0.14386
[12]	train-mlogloss:0.20750	eval-mlogloss:0.14228
[13]	train-mlogloss:0.20717	eval-mlogloss:0.14111
[14]	train-mlogloss:0.20663	eval-mlogloss:0.14070
[15]	train-mlogloss:0.20655	eval-mlogloss:0.14078
[16]	train-mlogloss:0.20623	eval-mlogloss:0.13866
[17]	train-mlogloss:0.20546	eval-mlogloss:0.13832
[18]	train-mlogloss:0.20522	eval-mlogloss:0.13679
[19]	train-mlogloss:0.20509	eval-mlogloss:0.13713
[20]	train

[I 2025-02-20 17:49:38,752] Trial 1 finished with value: 1.0 and parameters: {'lambda': 3.8343671630823825e-07, 'alpha': 5.930274879222062e-08, 'eta': 0.2773277890515301, 'gamma': 3.606256028971427e-07, 'max_depth': 6, 'min_child_weight': 9, 'subsample': 0.8798622924561192, 'colsample_bytree': 0.6919976935815868}. Best is trial 1 with value: 1.0.


...

[0]	train-mlogloss:0.96749	eval-mlogloss:0.97590
[1]	train-mlogloss:0.69902	eval-mlogloss:0.69222


[I 2025-02-20 17:50:03,698] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:0.91361	eval-mlogloss:0.91202
[1]	train-mlogloss:0.71454	eval-mlogloss:0.70263
[2]	train-mlogloss:0.57216	eval-mlogloss:0.55261


[I 2025-02-20 17:50:03,799] Trial 49 pruned. Trial was pruned at iteration 2.


Best trial: {'lambda': 3.8343671630823825e-07, 'alpha': 5.930274879222062e-08, 'eta': 0.2773277890515301, 'gamma': 3.606256028971427e-07, 'max_depth': 6, 'min_child_weight': 9, 'subsample': 0.8798622924561192, 'colsample_bytree': 0.6919976935815868}
Best accuracy: 1.0


In [63]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()